In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
df_aq = pd.read_csv("../input/beijing_1701_1803.csv")
df_station = pd.read_csv("../input/Beijing_AirQuality_Stations.csv")
target_cols = ['PM2.5', 'PM10', 'O3']
df_aq.utc_time = pd.to_datetime(df_aq.utc_time)

dfs = []
for stationId in df_aq.stationId.unique():
    df = df_aq[df_aq['stationId']==stationId]
    df = df.resample('1H', on='utc_time').sum()
    df = df.reset_index()
    df['stationId'] = stationId
    dfs.append(df)
df_origin = pd.concat(dfs)

# Station
idx_to_station = {k:v for k,v in df_station.Station_ID.iteritems()}
near_stations = []
for k, row in df_station.iterrows():
    result =  np.sqrt((111*(df_station.longitude-row[1]))**2 + (74*(df_station.latitude-row[2]))**2).nsmallest(4)
    near_idx = np.sqrt((111*(df_station.longitude-row[1]))**2 + (74*(df_station.latitude-row[2]))**2).nsmallest(4).index[1:4].tolist()
    near_station = [idx_to_station[idx] for idx in near_idx]
    near_stations.append(near_station)
    print(k, row[0])
    print(result, "\n")
    
df_station_near = pd.concat([df_station, pd.DataFrame(near_stations, columns=['near_1', 'near_2', 'near_3'])], axis=1 )

0 dongsi_aq
0     0.000000
30    3.300267
1     3.370048
4     4.506715
dtype: float64 

1 tiantan_aq
1     0.000000
31    1.621681
30    1.643067
0     3.370048
dtype: float64 

2 guanyuan_aq
2     0.000000
32    2.157452
3     4.040461
33    6.288367
dtype: float64 

3 wanshouxigong_aq
3     0.000000
33    2.409265
2     4.040461
31    4.664349
dtype: float64 

4 aotizhongxin_aq
4     0.000000
0     4.506715
32    5.716710
30    6.146011
dtype: float64 

5 nongzhanguan_aq
5     0.000000
34    2.446481
0     4.919748
1     7.083157
dtype: float64 

6 wanliu_aq
6     0.000000
2     7.192861
32    7.302417
8     8.949106
dtype: float64 

7 beibuxinqu_aq
7      0.000000
8      7.471527
17    11.267700
11    13.071216
dtype: float64 

8 zhiwuyuan_aq
8     0.000000
11    6.994566
7     7.471527
6     8.949106
dtype: float64 

9 fengtaihuayuan_aq
9    0.000000
3    8.178674
2    8.258877
6    9.218868
dtype: float64 

10 yungang_aq
10    0.000000
12    6.168689
11    7.883345
18    9.467663

In [3]:
# Spatial 
def spatial_step(df, target):
    for near in nears:
        df = pd.merge(df, df[[target, 'stationId','utc_time']], 
                     how='left', suffixes=('', '_'+ near),
                     left_on=['utc_time', near], 
                     right_on=['utc_time', 'stationId'])
    return df

# Temperial 
def temporal_step(df, target):
    dfs_ = []
    nulls = []
    for station in df.stationId.unique():
        df_ = df[df.stationId==station]
        df_ = df_.sort_values('utc_time')

        for i in [-2, -1, 1, 2]:
            df_[target + "_t{0:+}".format(i)] = df_[target].shift(i*-1)

        cols = [target + "_t{0:+}".format(i) for i in [-2, -1, 1, 2]]
        cols += [target+'_near_1', target+'_near_2', target+'_near_3', target, 'utc_time']

        df_[target+"_pred"] = df_[cols].mean(axis=1)

        # Update valie
        null_idxs = station_to_null_idxs[station]
        if EVALUATION:
            eval_idxs = station_to_eval_idxs[station]
        df_.loc[null_idxs, target] = df_[target + '_pred']
        if EVALUATION:
            df_.loc[eval_idxs, target] = df_[target + '_pred']

        for i in [-2, -1, 1, 2]:
            df_[target + "_t{0:+}".format(i)] = df_[target].shift(i*-1)
        null = df_[df_[target].isnull()].shape[0]
        nulls.append(null)
#         print(N_null, station, target)
        dfs_.append(df_[['utc_time', 'stationId', target]])
    return dfs_, nulls

df_stKNNs = []
EVALUATION = False
for target in target_cols:
    print(target)
    station_to_null_idxs = {}
    station_to_eval_idxs = {}
    MSEs = []

    nears = ['near_1', 'near_2', 'near_3']

    df_impute = pd.merge(df_origin, df_station_near, 'left', left_on='stationId', right_on='Station_ID')
    for station in df_impute.stationId.unique():
        df_ = df_impute[df_impute.stationId==station]
        df_ = df_.sort_values('utc_time')
        station_to_null_idxs[station] = df_[target].isnull() # for params update
        if EVALUATION:
            eval_condition = (df_.utc_time>='2017-05-01 00:00:00') & (df_.utc_time<='2017-05-01 23:00:00') & (df_.stationId=="tiantan_aq") 
            station_to_eval_idxs[station] = eval_condition # for evaluation

    N_nulls = 999
    while N_nulls>0:
        df_impute = spatial_step(df_impute, target)
        df_imputes, nulls = temporal_step(df_impute, target)
        df_impute = pd.concat(df_imputes)
        df_impute = pd.merge(df_impute, df_station_near, 'left', left_on='stationId', right_on='Station_ID')
        N_nulls = sum(nulls)
        print(N_nulls, nulls)
    
    df_stKNNs.append(df_impute)
    


18613 [527, 529, 544, 529, 524, 525, 525, 557, 525, 529, 529, 524, 530, 561, 524, 524, 548, 547, 526, 524, 524, 528, 527, 526, 531, 524, 535, 528, 531, 524, 526, 551, 529, 527, 551]
16119 [458, 458, 462, 461, 457, 457, 458, 479, 457, 459, 460, 457, 461, 479, 457, 457, 468, 461, 457, 457, 457, 458, 460, 458, 460, 457, 461, 457, 463, 457, 457, 477, 461, 459, 457]
14931 [426, 425, 427, 425, 425, 425, 426, 435, 425, 427, 428, 425, 428, 435, 425, 425, 427, 425, 425, 425, 425, 425, 428, 426, 425, 425, 428, 425, 431, 425, 425, 432, 425, 427, 425]
14205 [406, 405, 407, 405, 405, 405, 406, 410, 405, 405, 405, 405, 407, 411, 405, 405, 407, 405, 405, 405, 405, 405, 405, 406, 405, 405, 407, 405, 407, 405, 405, 411, 405, 405, 405]
13711 [392, 391, 392, 391, 391, 391, 391, 395, 391, 391, 391, 391, 393, 397, 391, 391, 393, 391, 391, 391, 391, 391, 391, 391, 391, 391, 393, 391, 393, 391, 391, 397, 391, 391, 391]
13291 [380, 379, 380, 379, 379, 379, 379, 383, 379, 379, 379, 379, 381, 385, 379, 379, 381

1901 [54, 54, 55, 54, 54, 54, 54, 54, 54, 54, 54, 54, 56, 56, 54, 54, 56, 54, 54, 54, 54, 54, 54, 54, 54, 54, 56, 54, 54, 54, 54, 56, 54, 54, 54]
1761 [50, 50, 51, 50, 50, 50, 50, 50, 50, 50, 50, 50, 52, 52, 50, 50, 52, 50, 50, 50, 50, 50, 50, 50, 50, 50, 52, 50, 50, 50, 50, 52, 50, 50, 50]
1621 [46, 46, 47, 46, 46, 46, 46, 46, 46, 46, 46, 46, 48, 48, 46, 46, 48, 46, 46, 46, 46, 46, 46, 46, 46, 46, 48, 46, 46, 46, 46, 48, 46, 46, 46]
1481 [42, 42, 43, 42, 42, 42, 42, 42, 42, 42, 42, 42, 44, 44, 42, 42, 44, 42, 42, 42, 42, 42, 42, 42, 42, 42, 44, 42, 42, 42, 42, 44, 42, 42, 42]
1341 [38, 38, 39, 38, 38, 38, 38, 38, 38, 38, 38, 38, 40, 40, 38, 38, 40, 38, 38, 38, 38, 38, 38, 38, 38, 38, 40, 38, 38, 38, 38, 40, 38, 38, 38]
1201 [34, 34, 35, 34, 34, 34, 34, 34, 34, 34, 34, 34, 36, 36, 34, 34, 36, 34, 34, 34, 34, 34, 34, 34, 34, 34, 36, 34, 34, 34, 34, 36, 34, 34, 34]
1061 [30, 30, 31, 30, 30, 30, 30, 30, 30, 30, 30, 30, 32, 32, 30, 30, 32, 30, 30, 30, 30, 30, 30, 30, 30, 30, 32, 30, 30, 30

4738 [134, 134, 135, 134, 136, 139, 134, 138, 134, 134, 134, 134, 137, 140, 134, 135, 139, 134, 134, 134, 137, 134, 136, 134, 134, 134, 136, 135, 136, 134, 137, 140, 134, 134, 136]
4458 [126, 126, 127, 126, 128, 131, 126, 130, 126, 126, 126, 126, 129, 132, 126, 127, 131, 126, 126, 126, 129, 126, 128, 126, 126, 126, 128, 127, 128, 126, 129, 132, 126, 126, 128]
4178 [118, 118, 119, 118, 120, 123, 118, 122, 118, 118, 118, 118, 121, 124, 118, 119, 123, 118, 118, 118, 121, 118, 120, 118, 118, 118, 120, 119, 120, 118, 121, 124, 118, 118, 120]
3898 [110, 110, 111, 110, 112, 115, 110, 114, 110, 110, 110, 110, 113, 116, 110, 111, 115, 110, 110, 110, 113, 110, 112, 110, 110, 110, 112, 111, 112, 110, 113, 116, 110, 110, 112]
3618 [102, 102, 103, 102, 104, 107, 102, 106, 102, 102, 102, 102, 105, 108, 102, 103, 107, 102, 102, 102, 105, 102, 104, 102, 102, 102, 104, 103, 104, 102, 105, 108, 102, 102, 104]
3338 [94, 94, 95, 94, 96, 99, 94, 98, 94, 94, 94, 94, 97, 100, 94, 95, 99, 94, 94, 94, 97, 94, 

7785 [222, 222, 223, 222, 222, 222, 222, 226, 222, 222, 222, 222, 222, 228, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 222, 224, 222, 222, 224, 222, 222, 222]
7505 [214, 214, 215, 214, 214, 214, 214, 218, 214, 214, 214, 214, 214, 220, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 214, 216, 214, 214, 216, 214, 214, 214]
7225 [206, 206, 207, 206, 206, 206, 206, 210, 206, 206, 206, 206, 206, 212, 206, 206, 206, 206, 206, 206, 206, 206, 206, 206, 206, 206, 206, 206, 208, 206, 206, 208, 206, 206, 206]
6945 [198, 198, 199, 198, 198, 198, 198, 202, 198, 198, 198, 198, 198, 204, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 198, 200, 198, 198, 200, 198, 198, 198]
6665 [190, 190, 191, 190, 190, 190, 190, 194, 190, 190, 190, 190, 190, 196, 190, 190, 190, 190, 190, 190, 190, 190, 190, 190, 190, 190, 190, 190, 192, 190, 190, 192, 190, 190, 190]
6385 [182, 182, 183, 182, 182, 182, 182, 186, 182, 182, 182, 182, 182, 188, 182, 182, 182, 182,

## Merge & 检查是否正确

In [4]:
df = pd.merge(df_origin, df_stKNNs[0][['utc_time', 'stationId', 'PM2.5']], how='left', suffixes=("", "_pred"),
                  left_on=['utc_time', 'stationId'],
                  right_on=['utc_time', 'stationId'],
             )

df = pd.merge(df, df_stKNNs[1][['utc_time', 'stationId', 'PM10']], how='left', suffixes=("", "_pred"),
                  left_on=['utc_time', 'stationId'],
                  right_on=['utc_time', 'stationId'],
             )

df = pd.merge(df, df_stKNNs[2][['utc_time', 'stationId', 'O3']], how='left', suffixes=("", "_pred"),
                  left_on=['utc_time', 'stationId'],
                  right_on=['utc_time', 'stationId'],
             )

tmp = df[pd.notnull(df[target])]
mean_absolute_error(tmp[target], tmp[target+'_pred'])

0.0

## Evalution (手动选择某一站点，设置一天空值)

In [ ]:
# condition = (df.stationId=="tiantan_aq") & (df.utc_time>='2017-05-01 00:00:00') & (df.utc_time<='2017-05-01 23:00:00')
# tmp = df[['utc_time','PM10', 'PM10_pred']][condition]
# tmp = tmp.sort_values(by='utc_time')
# # tmp
# tmp.plot(x='utc_time', y=['PM10', 'PM10_pred'], figsize=(16,6), style={'PM10': '-', 'PM10_pred': ':'})

# tmp = df[condition]
# tmp = tmp[pd.notnull(tmp[target])]
# MAE = mean_absolute_error(tmp[target], tmp[target+'_pred'])
# print(MAE)

In [6]:
for target in target_cols:
    df[target] = df[target + '_pred']

In [9]:
df = df[['utc_time', 'PM2.5', 'PM10', 'O3', 'stationId']]

In [10]:
df

,utc_time,PM2.5,PM10,O3,stationId
0,2017-01-01 22:00:00,453.000000,467.000000,3.000000,aotizhongxin_aq
1,2017-01-01 23:00:00,417.000000,443.000000,2.000000,aotizhongxin_aq
2,2017-01-02 00:00:00,395.000000,467.000000,3.000000,aotizhongxin_aq
3,2017-01-02 01:00:00,420.000000,484.000000,3.000000,aotizhongxin_aq
4,2017-01-02 02:00:00,453.000000,520.000000,4.000000,aotizhongxin_aq
5,2017-01-02 03:00:00,429.000000,470.998476,3.000000,aotizhongxin_aq
6,2017-01-02 04:00:00,211.000000,451.306961,7.000000,aotizhongxin_aq
7,2017-01-02 05:00:00,116.000000,424.100020,4.000000,aotizhongxin_aq
8,2017-01-02 06:00:00,51.000000,401.935667,26.000000,aotizhongxin_aq
9,2017-01-02 07:00:00,38.000000,380.150756,28.000000,aotizhongxin_aq
